In [1]:
!cd '/Users/clairegong/Downloads/ACV Project'
!pwd
root = '/Users/clairegong/Downloads/ACV Project'

/Users/clairegong/Downloads/ACV Project


In [2]:
from celeba import CelebA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from random import randint


Bad key savefig.frameon in file /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably need to get an updated matplotlibrc fil

## 1. check data

In [3]:
#check private data size are also aligned, no need to resize
private_set = datasets.ImageFolder(root, transform=transforms.ToTensor())
private_loader = torch.utils.data.DataLoader(private_set, shuffle=False, batch_size=32)
img, lb = iter(private_loader).next()
print(img.size())

torch.Size([32, 3, 218, 178])


In [24]:
# Data loading code
#1. all mages are already aligned to 218*178;

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = CelebA(
    root,
    'train600.txt',
    transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
#         transforms.Resize((178,178)),
#         transforms.CenterCrop((178,178)),
        transforms.RandomResizedCrop(178, scale=(0.8,1.0)),# should not cut many info for multi label classification
        transforms.ToTensor(),
        normalize,
    ]))

val_dataset = CelebA(
    root,
    'val200.txt',
    transforms.Compose([
        transforms.Resize((178,178)),
#         transforms.CenterCrop((178,178)),
        transforms.ToTensor(),
        normalize,
    ]))

test_dataset = CelebA(
    root,
    'test200.txt',
    transforms.Compose([
        transforms.Resize((178,178)),
#         transforms.CenterCrop((178,178)),
        transforms.ToTensor(),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [25]:
def check_normalize(train_loader):
    mean = 0.
    std = 0.
    for images, _ in test_loader:
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(train_loader.dataset)
    std /= len(train_loader.dataset)
    print(mean)
    print(std)
check_normalize(train_loader)

tensor([ 0.0537, -0.0324, -0.0333])
tensor([0.3751, 0.3507, 0.3415])


### check images

In [ ]:

def show_random_image(loader):
    img, label = next(iter(loader))
    idx = randint(0,loader.batch_size-1)
    img = np.transpose(img[idx].squeeze(),(1,2,0))
    print(img.size())
    plt.imshow(img)
    plt.show()

In [ ]:
show_random_image(train_loader)
show_random_image(val_loader)
show_random_image(test_loader)

## 2. load pretrained model

### Resnet18: freeze conv layers

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_pt = models.resnet18(pretrained=True)
in_size = model_pt.fc.in_features#512->1000 in-out size

#freeze pretrained parameters
for param in model_pt.parameters():
    param.requires_grad = False

#we need 512->2 out size
# Parameters of newly constructed modules have requires_grad=True by default
# model_pt.fc = nn.Sequential(nn.Dropout(p=0.2),nn.Linear(in_size,256), nn.ReLU(inplace=True), nn.Linear(256,40))
model_pt.fc = nn.Linear(in_size,40)
model_pt = model_pt.to(device)

#check model paramters flexbility
print(model_pt.conv1.weight.requires_grad)

# checkpoint0 = torch.save(model_pt.state_dict(),'checkpoint0.pth')
state_dict0 = copy.deepcopy(model_pt.state_dict())

False


In [19]:
# model_pt

### Resnet50

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_pt = models.resnet18(pretrained=True)
in_size = model_pt.fc.in_features#512->1000 in-out size

#freeze pretrained parameters
for param in model_pt.parameters():
    param.requires_grad = False

#Parameters of newly constructed modules have requires_grad=True by default


# model_pt.fc = nn.Sequential(nn.Dropout(p=0.2),nn.Linear(in_size,512), \
#                             nn.Dropout(p=0.2),nn.ReLU(inplace=True), nn.Linear(512,40))

model_pt.fc = nn.Sequential(nn.Dropout(p=0.5),nn.Linear(in_size,40))
model_pt = model_pt.to(device)

#check model paramters flexbility
print(model_pt.conv1.weight.requires_grad)

# checkpoint0 = torch.save(model_pt.state_dict(),'checkpoint0.pth')
state_dict0 = copy.deepcopy(model_pt.state_dict())

In [ ]:
model_pt.fc

### (optional)sklearn multi class accuracy

In [ ]:
##sklearn accuracy for multi class is by sample, only all correct class prediction for a sample is True
x = torch.Tensor([[0,1,1,0],[1,1,1,1]])
print('x \n',x)
y = torch.Tensor([[0,1,0,0],[1,1,1,1]])
print('y \n',y)
from sklearn.metrics import accuracy_score
acc = accuracy_score(x,y)
print('sklearn accuracy: ',acc)#1/2

##test my accuracy function logic
scores = torch.randn(2,4)
preds = torch.sigmoid(scores).round()
print('prediction\n',preds)
truth = (preds==y)#stupid typo here
print('truth table\n',truth)
no_correct = truth.float().sum().item()
no_correct_attr = truth.float().sum(0)#40-size tensor
print( no_correct, no_correct_attr  )

## 3. training , saving and testing

In [ ]:
def batch_accuracy(scores, labels):
    preds = torch.sigmoid(scores).round()
    truth = (preds==labels)
    num_correct = truth.float().sum().item()/40 #avg no of  correct prediction * no of samples
    num_correct_attr = truth.float().sum(0)#40-size tensor
    return num_correct, num_correct_attr  

In [ ]:


def train_model(model, criterion, optimizer,scheduler, epoches=1, test=True):
    start = time.time()
    
    best_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(0, epoches):

        running_loss = train_acc = 0

        
        for x,y in train_loader:
            x,y = x.to(device), y.to(device)
            
            model.train()
            optimizer.zero_grad()
            
            x.requires_grad_()
            s = model(x)
            loss = criterion(s,y.float())
            loss.backward()
            optimizer.step()
            
            running_loss += loss.detach().item()*x.size(0)
            train_acc += batch_accuracy(s.detach(),y)[0]
        running_loss /= len(train_dataset)
        train_acc /= len(train_dataset)
        print('epoch',epoch+1,'/',epoches, 'time elapsed:%d s '%( time.time()-start))        
        print('running loss:', running_loss, 'training accuracy:' , train_acc*100)
        
        scheduler.step()
        
        if epoch%3 == 0 and test:
            with torch.no_grad():
                running_loss = val_acc = 0
                for x,y in val_loader:
                    x,y = x.to(device), y.to(device)
                    model.eval()
                    s = model(x)
                    loss = criterion(s,y.float())

                    running_loss += loss.detach().item()*x.size(0)
                    val_acc += batch_accuracy(s.detach(),y)[0]
                running_loss /= len(val_dataset)
                val_acc /= len(val_dataset)
                print('validation loss:', running_loss, 'validation accuracy:' , val_acc*100)
                    
        #copy model weights if test accuracy improves
        if val_acc>best_acc:
            best_acc = val_acc
            best_weights = copy.deepcopy(model.state_dict())
        
    
    print('best accuracy', best_acc*100)

    #load best weights
    model.load_state_dict(best_weights)
    return model            
    

In [ ]:
def test_evaluation(model, loader=test_loader):

    start = time.time()
    with torch.no_grad():
        running_loss = test_accuracy = 0
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            model.eval()
            s = model(x)
            loss = criterion(s,y.float())

            running_loss += loss.detach().item()*x.size(0)
            test_accuracy += batch_accuracy(s.detach(),y)[0]
        running_loss /= len(loader.dataset)
        test_accuracy /= len(loader.dataset)
        print('time elapsed:%d s '%( time.time()-start)) 
        print('test loss:', running_loss, 'test accuracy:' , test_accuracy*100)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
#only optimize new fc layer
optimizer = optim.SGD(model_pt.fc.parameters(), lr = 0.001, momentum=0.9)
optimizer_conv = optim.SGD(model_pt.parameters(), lr = 0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#train
model_pt.load_state_dict(state_dict0)
model = train_model(model_pt, criterion, optimizer_conv, scheduler, epoches=10)
# save
state_dict = torch.save(model.state_dict(),'checkpoint.pth')
# test
test_evaluation(model)

In [ ]:
# list(model.fc.parameters())

In [ ]:
optimizer.state_dict()

In [ ]:
optimizer.state_dict()['param_groups'][0]['lr'] = 0.01

In [ ]:
# model_pt.load_state_dict(state_dict0)
# list(model_pt.fc.parameters())

## 4. Parameter tuning
### experiments i

### No. 1: epoch=1
- batch_size:32
- data augmentation:random horizontal flip
- model: resnet18
- optimizer: SGD+schedueler, lr=0.001
- freeze conv layers: True
- fc layer: nn.Linear(512,40)
- epoches: 1
- time: 79s
- evaluation: <br>test loss: 0.48917443871498106 test accuracy: 79.7625


### No. 2: epoch =10
- batch_size:32
- data augmentation:random horizontal flip
- model: resnet18
- optimizer: SGD+schedueler, lr=0.001
- freeze conv layers: True
- fc layer: nn.Linear(512,40)
- epoches: 10
- time: 79s
- evaluation: <br>test loss: 0.4280791974067688 test accuracy: 81.625


### No. 3: learning rate 0.01,test>>train? random...
- batch_size:32
- data augmentation:random horizontal flip
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: nn.Linear(512,40)
- epoches: 1
- time: 13min
- evaluation: <br>
epoch 1 / 1 time elapsed:75 s 
running loss: 0.5641420304775238 training accuracy: 70.19999999999999
validation loss: 0.44201597809791565 validation accuracy: 80.375
best accuracy 80.375
test loss: 0.4328741908073425 test accuracy: 81.5625


### No. 4: Data augmentation: rotation + centercrop
- batch_size:32
- data augmentation:random horizontal flip +        transforms.RandomRotation(20),    transforms.CenterCrop((178,178)),
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: nn.Linear(512,40)
- epoches: 10
- time: 13min
- evaluation: <br>test loss: 0.43850406527519226 test accuracy: 81.3875
- comment: 1st epoch acc: 50%, after 4 epoch reach 80%, then stable...

### No. 5: Data augmentation: rotation + randomcrop, similar to no.5
- batch_size:32
- data augmentation:random horizontal flip +        transforms.RandomRotation(20),    transforms.CenterCrop((178,178)),
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: nn.Linear(512,40)
- epoches: 10
- time: 13min
- evaluation: <br>test loss: 0.44117275953292845 test accuracy: 81.30000000000001
- comment: 1st epoch acc: 50%, after 4 epoch reach 80%, then stable...


## Experiment ii retrain resnet18+1fc is  best

### No. 6: retrain conv, much better! save as checkpoint6.pth (best efficiency)
### Means 1fc layer lacks complexity
- batch_size:32
- data augmentation:random horizontal flip +        transforms.RandomRotation(20),    transforms.CenterCrop((178,178)),
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: nn.Linear(512,40)
- epoches: 10
- time: 13min++
- evaluation: 1st epoch: validation loss: 0.4332087850570679 validation accuracy: 81.2625

test loss: 0.30936500668525696 test accuracy: 86.775

### No. 7: pretrained+deep 2 fc layers +drop out
- batch_size:32
- data augmentation:random horizontal flip +        transforms.RandomRotation(20),    transforms.CenterCrop((178,178)),
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: model_pt.fc = nn.Sequential(nn.Dropout(p=0.2),nn.Linear(in_size,256), nn.ReLU(), nn.Linear(256,4))
- epoches: 10&20
- time: 15min+ 23min
- evaluation: <br> 10epoches: test loss: 0.39907381772994993 test accuracy: 83.71249999999999
- 20epoches: test loss: 0.3699471116065979 test accuracy: 83.9875

### No. 8: resnet50 +1fc
- batch_size:32
- data augmentation:random horizontal flip +        transforms.RandomRotation(20),    transforms.CenterCrop((178,178)),
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: nn.Linear(2048,40)
- epoches: 3/10
- time: 4min/epoch
- evaluation: <br>
epoch 1 / 10 time elapsed:235 s 
running loss: 0.5192274848620096 training accuracy: 74.84583333333333
validation loss: 0.43311495304107667 validation accuracy: 80.6
epoch 2 / 10 time elapsed:551 s 
running loss: 0.4324911272525787 training accuracy: 81.04166666666666
epoch 3 / 10 time elapsed:814 s 

running loss: 0.4192929100990295 training accuracy: 82.25833333333334

### No. 9: resnet50 +dropout(0.5)+1fc
- batch_size:32
- data augmentation:random horizontal flip +        transforms.RandomRotation(20),    transforms.CenterCrop((178,178)),
- optimizer: lr=0.01
- model: resnet18
- freeze conv layers: True
- fc layer: nn.Linear(2048,40)
- epoches: 10
- time: 5.5min/epoch
- evaluation: <br>
epoch 1 / 10 time elapsed:332 s 
running loss: 0.5315309766928354 training accuracy: 73.51666666666668
validation loss: 0.4316992115974426 validation accuracy: 81.03750000000002
epoch 2 / 10 time elapsed:778 s 
running loss: 0.4411350138982137 training accuracy: 80.71666666666665
epoch 3 / 10 time elapsed:1549 s 
running loss: 0.4266382654507955 training accuracy: 81.64166666666665

## 5. Generating predictions.txt

In [ ]:
final_data = datasets.ImageFolder(root+'/data/', transform = transforms.Compose([
        transforms.Resize((178,178)),
        transforms.ToTensor(),
        normalize]))
final_loader = torch.utils.data.DataLoader(final_data, shuffle=False, batch_size=32)

In [ ]:
show_random_image(final_loader)

In [ ]:
def test_attributes(model, loader):
    state_dict = torch.load('checkpoint6.pth')
    model.load_state_dict(state_dict)
    start = time.time()
    with torch.no_grad():
        running_loss = test_accuracy = 0
        attr_acc = torch.Tensor(40)
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            model.eval()
            s = model(x)
            loss = criterion(s,y.float())

            running_loss += loss.detach().item()*x.size(0)
            test_accuracy += batch_accuracy(s.detach(),y)[0]
            attr_acc += batch_accuracy(s.detach(),y)[1]
        running_loss /= len(loader.dataset)
        test_accuracy /= len(loader.dataset)
        attr_acc /= len(loader.dataset)        
        print('time elapsed:%d s '%( time.time()-start)) 
        print('test loss:', running_loss, 'test accuracy:' , test_accuracy*100)
        print('accuracy per attribute', attr_acc)
        
        return attr_acc
attr_acc = test_attributes(model_pt, test_loader) 

In [ ]:
rank_attributes = np.argsort(attr_acc)
rank_attributes

In [ ]:
def output_labels(model, loader):
#     state_dict = torch.load('checkpoint0.pth')
#     model.load_state_dict(state_dict)
    
    with torch.no_grad():
        running_loss = test_accuracy = 0
        arr = torch.Tensor((1,40))
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            model.eval()
            s = model(x)
            preds = torch.sigmoid(s).round()# prob 0.5 is 1
            preds = preds*2 - 1
            arr = np.concatenate((arr,preds), axis=0)    
    return arr


### make prediction files

In [ ]:

import os

labels = output_labels(model_pt, final_loader)
np.savetxt("predictions.txt", arr, fmt='%d', footer='\n', comments='')

test_images = '/Users/clairegong/Downloads/ACV Project/data/test_img'
img_names = os.listdir(test_images)
file = "predictions.txt"

with open(file, 'r') as f:
    lines = [' '.join([img, x.strip(), '\n']) for x,img in zip(f.readlines(),img_names)]

with open(file, 'w') as f:
    f.writelines(lines) 

### check data positive attribute distribution

In [ ]:
#check sample labels for e.g. attribute idx 35 (0-39)
attr35 = np.array(test_dataset.targets)[:,35]
idx = list(np.where(attr35==1)[0].astype(int))
idx

## try to take section of data with multiple indices, but can not, dataset is in the form of a list
# torch.utils.data.DataLoader(test_dataset[idx])

In [ ]:
def positive_attributes(dataset):
    #dim tip, arr dim is (200,40), put 0/row then calculation cross 200 rows, result size will be same as dim 1-> 40
    distribution = np.array(dataset.targets).sum(0)
    sort_idx = np.argsort(distribution)+1
    print('for {} samples, min num {} for attribute {} and max num {} for attribute {}'.format(len(dataset),distribution.min(), sort_idx[0],\
                                                                               distribution.max(), sort_idx[-1]))
    return sort_idx
#     return distribution

print(positive_attributes(train_dataset))
print(positive_attributes(val_dataset))
print(positive_attributes(test_dataset))

##check no of labels per person have
# plt.hist(np.array(train_dataset.targets).sum(1),bins=17);

## reference: attribute names

In [ ]:
import requests
import IPython.display as Disp
url = 'https://www.researchgate.net/publication/327029519/figure/tbl1/AS:667628372766726@1536186416689/List-of-the-40-face-attributes-provided-with-the-CelebA-database.png'
Disp.Image(requests.get(url).content)


## visualizing test accuracies

In [4]:
import pandas as pd
import numpy as np

In [60]:
attributes = pd.read_csv('/Users/clairegong/Downloads/list_attr_celeba.txt', sep=' ', header=None)
accuracies = [0.9482, 0.8340, 0.8293, 0.8538, 0.9893, 0.9600, 0.7152, 0.8398, 0.9002,
        0.9596, 0.9632, 0.8904, 0.9260, 0.9589, 0.9639, 0.9962, 0.9744, 0.9833,
        0.9184, 0.8758, 0.9813, 0.9377, 0.9696, 0.8766, 0.9619, 0.7556, 0.9706,
        0.7684, 0.9381, 0.9504, 0.9775, 0.9283, 0.8448, 0.8488, 0.9031, 0.9904,
        0.9385, 0.8739, 0.9682, 0.8886]
attributes.loc[1] = accuracies
attributes = attributes.T
attributes.columns=['attr','acc']
attributes['acc'] = attributes['acc'].astype('float64') *100

In [6]:
# attributes = attributes.sort_values(by='acc', ascending=False).round(1).reset_index().drop(['index'],axis=1)

In [91]:
# di = dict(zip(attributes['attr'], attributes['acc']))

## check predictions.txt

In [216]:
import pandas as pd
path1 = '/Users/clairegong/Downloads/predictions_1020_2am.txt'
path2 = '/Users/clairegong/Downloads/predictions_1021_2pm.txt'
path3 = '/Users/clairegong/Downloads/predictions_1021_4pm.txt'#removed space in last col
ref1 = '/Users/clairegong/Downloads/prediction_utsa.txt'
ref2 = '/Users/clairegong/Downloads/predictions_vakul.txt'

In [217]:
pred1 = pd.read_csv(path1, sep=' ', header=None).drop([41], axis=1)
pred2 = pd.read_csv(path2, sep=' ', header=None).drop([41], axis=1)
pred3 = pd.read_csv(path3, sep=' ', header=None)
ref1 = pd.read_csv(ref1, sep=' ', header=None)
ref2 = pd.read_csv(ref2, sep=' ', header=None)

In [227]:
#with utsa
(pred2==ref1).mean().mean()
(pred3==ref1).mean().mean()

0.8914520793360281

In [226]:
#with vakul
(pred2==ref2).mean().mean()
(pred2==ref2).mean().mean()

0.8540179484769229

In [213]:
x = 1
print(str(x)+'\n')

1



### visual check

In [63]:
attr = list(attributes['attr'])
attr.insert(0, 'img')
pred.columns=attr

In [105]:
# check image labels
name = 'Bridgette_Wilson-Sampras_0001.jpg'
truth = pred[pred.img==name].T == 1
pred[pred.img==name].T[truth].dropna()

,1541
Blond_Hair,1
No_Beard,1
Smiling,1
Wavy_Hair,1
Wearing_Lipstick,1
Young,1
